Instead of only following 1 condition this version of the stock screener will check 8 conditions before writing the *up and coming* stocks to an excel file

This data in the excel file will then be upploaded to your aws db creating a serverless architecture for your application. 

CONDITIONS:


---

1: current price > 150 SMA & > 200 SMA

2: 150 SMA and > 200 SMA

3: 200 SMA trending up for at least 1 month

4: 50 SMA> 150 SMA and 50 SMA > 200 SMA 

5: Current Price > 50 SMA

6: Current Price is at least 30% above 52 week low 

7: Current Price is within 25% of 52 week high

8: IBD RS rating >70 and the higher the better
	



In [ ]:
#install yf 
!pip install yfinance

In [ ]:
#import libs
import datetime as dt 
import pandas as pd 
from pandas_datareader import data as pdr
import yfinance as yf
from tkinter import Tk
import os 
from google.colab import files
from pandas import ExcelWriter

In [ ]:
yf.pdr_override()
start = dt.datetime(2017,12,1)
now = dt.datetime.now()
df = pdr.get_data_yahoo(stock,start,now)

[*********************100%***********************]  1 of 1 completed


In [ ]:
#upload file with stock ticker and RS
uploaded = files.upload()

Saving RichardStocks.xlsx to RichardStocks (1).xlsx


In [ ]:
#stockList = data sheet
stocklist = pd.read_excel('/content/RichardStocks.xlsx')

#exportlist
#defining the cols for the csv file to be exported 
exportlist = pd.DataFrame(columns = ['Stock',"RS_Rating", "50 Day MA", "150 Day MA", 
                                     "200 Day MA", "52 Week Low", "52 week High"])

#searching yahoo finance for data on stocks in the file
for i in stocklist.index:
  stock= str(stocklist["Symbol"][i])
  RS_Rating = int(stocklist["RS Rating"][i])

  sma = [50,150,200]

  for x in sma:
    sma = x
    df["SMA_"+str(sma)]=round(df.iloc[:,4].rolling(window=sma).mean(),2)
      
    #last availabe record on yahoo finance
    currentClose=df["Adj Close"][-1]
      
    # 50 day moving avg
    moving_average_50=df["SMA_50"][-1]
      
    #150 day moving avg
    moving_average_150=df["SMA_150"][-1]
      
    #200 moving avg
    moving_average_200=df["SMA_200"][-1]
      
    #getting the highs and lows of a 52 week period
    low_of_52week=min(df["Adj Close"][-260:])
    high_of_52week=max(df["Adj Close"][-260:])

    #checking that the 200 day has been treding for a min month of trading
    try:
		    moving_average_200_20 = df["SMA_200"][-20]
    except Exception:
		   moving_average_200_20=0

  print("Checking "+stock)
    
  #condition 2: 150 day > 200 day
  if (moving_average_150 > moving_average_200):
    cond_2 == True
  else:
    print("Failed cond 2 ")

  #condition 5: Current Price > 50 SMA
  if (currentClose > moving_average_50):
    cond_5 == True
  else:
    print("Failed cond 5 ")
  #condition 6: Current Price is at least 30% above 52 week low 
  # current price greater than 1.3 * low_52
  if (currentClose >= (1.3 * low_of_52week)):
    cond_6 == True
  else:
    print("Failed cond 6 ")
  #condiition 8: RS rating > 70 
  if (RS_Rating > 70 ):
    cond_8 == True
  else: 
    print("Failed cond 8 ")  
  
  if (cond_2 and cond_5 and cond_6 and cond_8 == True):      
    exportlist = exportlist.append({'stock' :stock, "RS rating" : RS_Rating , "50 day MA" : moving_average_50,"150 Day MA": moving_average_150, "200 Day MA": moving_average_200,"52 Week Low": low_of_52week, "52 week High": high_of_52week},ignore_index=True)
  else:     
    print("No data on "+stock)
print(exportlist)   

In [ ]:
print (exportlist)

      Stock  RS_Rating  50 Day MA  ...  50 day MA  RS rating  stock
0       NaN        NaN        NaN  ...      29.65       99.0   INMD
1       NaN        NaN        NaN  ...      29.65       99.0   ENPH
2       NaN        NaN        NaN  ...      29.65       98.0   PLMR
3       NaN        NaN        NaN  ...      29.65       95.0   VCTR
4       NaN        NaN        NaN  ...      29.65       98.0   SEDG
...     ...        ...        ...  ...        ...        ...    ...
4455    NaN        NaN        NaN  ...      29.65       16.0   UTSI
4456    NaN        NaN        NaN  ...      29.65       11.0   MKGI
4457    NaN        NaN        NaN  ...      29.65       11.0    JMP
4458    NaN        NaN        NaN  ...      29.65       16.0   INSE
4459    NaN        NaN        NaN  ...      29.65        9.0    NTN

[4460 rows x 10 columns]


In [ ]:
exportlist.to_csv('stocks.csv',index=False)


In [ ]:

  #condition 2: 150 day > 200 day
  if (moving_average_150 > moving_average_200):
    cond_2 = True
  else:
    print("Failed cond 2 ")

  #condition 5: Current Price > 50 SMA
  if (currentClose > moving_average_50):
    cond_5 = True
  else:
    print("Failed cond 5 ")
  #condition 6: Current Price is at least 30% above 52 week low 
  # current price greater than 1.3 * low_52
  if (currentClose >= (1.3 * low_of_52week)):
    cond_6 = True
  else:
    print("Failed cond 6 ")
  #condition 7: Current Price is within 25% of 52 week high
  if (currentClose >= (.75 * high_of_52week)):
    cond_7 = True
  else:
    print("Failed cond 7 ")
  #condiition 8: RS rating > 70 
  if (RS_Rating > 70 ):
    cond_8 = True
  else: 
    print("Failed cond 8 ")

   if (cond_1 and cond_2 and cond_3 and cond_4 and cond_5 and cond_6 and cond_7 and cond_8):  